# Hands-on data engineering Vantage AI

Deze sessie gaan we neurale netwerken trainen om simpele images te classificeren van de CIFAR-10 dataset. 

## Dependency management
Deze notebook gaat er vanuit dat je de volgende python dependencies geïnstalleerd hebt:
- Jupyter
- Tensorflow
- Keras
- Matplotlib
- SKLearn

Opdracht: _Schrijf een `requirements.txt` waarmee de requirements van deze notebook makkelijk geïnstalleerd kunnen worden._

## Data inladen

De data bestaat uit 3 delen: train, validatie en test set.

Opdracht: _Er is veel herhaling in deze code. Splits dit op in leesbare en herbruikbare code. Denk hierbij aan de engineering principes die we hebben behandeld._

In [1]:
import numpy as np
import os
import tarfile
from urllib.request import urlretrieve
import pickle
import random


def load_data():
    # training set, batches 1-4
    if not os.path.exists(os.path.join(os.getcwd(), "data")):
        os.makedirs(os.path.join(os.getcwd(), "data"))

        
    dataset_dir = os.path.join(os.getcwd(), "data")
    
    if not os.path.exists(os.path.join(dataset_dir, "cifar-10-batches-py")):
        print("Downloading data...")
        urlretrieve("http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", os.path.join(dataset_dir, "cifar-10-python.tar.gz"))
        tar = tarfile.open(os.path.join(dataset_dir, "cifar-10-python.tar.gz"))
        tar.extractall(dataset_dir)
        tar.close()
        
    train_X = np.zeros((40000, 3, 32, 32), dtype="float32")
    train_y = np.zeros((40000, 1), dtype="ubyte").flatten()
    n_samples = 10000  # aantal samples per batch
    dataset_dir = os.path.join(dataset_dir,"cifar-10-batches-py")
    for i in range(0,4):
        f = open(os.path.join(dataset_dir, "data_batch_"+str(i+1)), "rb")
        cifar_batch = pickle.load(f,encoding="latin1")
        f.close()
        train_X[i*n_samples:(i+1)*n_samples] = (cifar_batch['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
        train_y[i*n_samples:(i+1)*n_samples] = np.array(cifar_batch['labels'], dtype='ubyte')

    # validation set, batch 5
    f = open(os.path.join(dataset_dir, "data_batch_5"), "rb")
    cifar_batch_5 = pickle.load(f,encoding="latin1")
    f.close()
    val_X = (cifar_batch_5['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    val_y = np.array(cifar_batch_5['labels'], dtype='ubyte')

    # labels
    f = open(os.path.join(dataset_dir, "batches.meta"), "rb")
    cifar_dict = pickle.load(f,encoding="latin1")
    label_to_names = {k:v for k, v in zip(range(10), cifar_dict['label_names'])}
    f.close()

    # test set
    f = open(os.path.join(dataset_dir, "test_batch"), "rb")
    cifar_test = pickle.load(f,encoding="latin1")
    f.close()
    test_X = (cifar_test['data'].reshape(-1, 3, 32, 32) / 255.).astype("float32")
    test_y = np.array(cifar_test['labels'], dtype='ubyte')


    print("training set size: data = {}, labels = {}".format(train_X.shape, train_y.shape))
    print("validation set size: data = {}, labels = {}".format(val_X.shape, val_y.shape))
    
    print("Test set size: data = "+str(test_X.shape)+", labels = "+str(test_y.shape))

    return train_X, train_y, val_X, val_y, test_X, test_y, label_to_names


### Preprocessing
Bij CIFAR10 is er niet veel preprocessing nodig. Normalisatie van de data is vaak een goed idee, vantevoren berekenen we de gemiddelde pixelwaarde en bij het batchgewijs trainen normaliseren we de data aan de hand van die waarde. Het is een goed idee om deze mean in een pickle bestand op te slaan, en die dan in te laden bij het opstarten zodat voor predicten niet de hele dataset nodig is. 

In [2]:
nr_channels = 3
image_size = 32
nr_classes = 10
epochs = 20

train_X, train_y, val_X, val_y, test_X, test_y, label_to_names = load_data()

# Conv nets trainen duurt erg lang op CPU, dus we gebruiken maar een klein deel
# van de data nu, als er tijd over is kan je proberen je netwerk op de volledige set te runnen
train_X = train_X[:10000]
train_y = train_y[:10000]

def calc_mean_std(X):
    mean = np.mean(X)
    std = np.std(X)
    return mean, std

def normalize(data, mean, std):
    return (data-mean)/std

#De data van train_X is genoeg om de mean en std van de hele set nauwkeurig te benaderen
mean,std = calc_mean_std(train_X)
test_X = normalize(test_X,mean,std)
val_X = normalize(val_X,mean,std)
train_X = normalize(train_X ,mean,std)


training set size: data = (40000, 3, 32, 32), labels = (40000,)
validation set size: data = (10000, 3, 32, 32), labels = (10000,)
Test set size: data = (10000, 3, 32, 32), labels = (10000,)


# Definieer model
We gebruiken de volledige images om een convolutioneel neuraal netwerk te definieren en te trainen. Alhoewel de data science niet de focus heeft in deze cursus is het wel belangrijk om te begrijpen wat er gebeurt, dus schroom niet ook vragen te stellen over het model.

In [3]:
from keras.models import Model
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, Input
from sklearn.metrics import classification_report

def conv_net():
    # We definieren de input van het netwerk als de shape van de input,
    # minus de dimensie van het aantal plaatjes, uiteindelijk dus (3, 32, 32).
    input = Input(shape=train_X.shape[1:])
    
    # Eerste convolutielaag
    # Padding valid betekent dat we enkel volledige convoluties gebruiken, zonder padding
    # Data format channels_first betekent dat de channels eerst komen, en dan pas de size van ons plaatje
    # Dus (3, 32, 32) in plaats van (32, 32, 3)
    conv = Conv2D(filters=16, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu')(input)
    
    # Nog een convolutielaag, dit keer met stride=2 om de inputsize te verkleinen
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='valid',
                  data_format='channels_first', activation='relu', strides=(2, 2))(conv)
    
    #Voeg een flatten laag toe, om te schakelen naar de dense layer
    flatten = Flatten()(conv)
   
    # De softmax laag voor de probabilities 
    output_layer = Dense(units=nr_classes, activation='softmax')(flatten)
    
    model = Model(inputs=input, outputs=output_layer)
    
    # Het model moet nog gecompiled worden en loss+learning functie gespecificeerd worden
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


model = conv_net()

model.fit(x=train_X, y=train_y, batch_size=50, epochs=10, validation_data=(val_X, val_y), verbose=2)
predictions = np.array(model.predict(test_X, batch_size=100))
test_y = np.array(test_y, dtype=np.int32)
#Take the highest prediction
predictions = np.argmax(predictions, axis=1)

#Print resultaten
print("Accuracy = {}".format(np.sum(predictions == test_y) / float(len(predictions))))
print(classification_report(test_y, predictions, target_names=list(label_to_names.values())))

Using TensorFlow backend.
C:\Users\Railway\AppData\Local\conda\conda\envs\ds-python\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Railway\AppData\Local\conda\conda\envs\ds-python\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Railway\AppData\Local\conda\conda\envs\ds-python\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 10000 samples
Epoch 1/10
 - 9s - loss: 1.7454 - acc: 0.3826 - val_loss: 1.5579 - val_acc: 0.4536
Epoch 2/10
 - 8s - loss: 1.3389 - acc: 0.5289 - val_loss: 1.4517 - val_acc: 0.4948
Epoch 3/10
 - 8s - loss: 1.1369 - acc: 0.6044 - val_loss: 1.3849 - val_acc: 0.5110
Epoch 4/10
 - 8s - loss: 0.9780 - acc: 0.6646 - val_loss: 1.4341 - val_acc: 0.5144
Epoch 5/10
 - 8s - loss: 0.8401 - acc: 0.7073 - val_loss: 1.4693 - val_acc: 0.5083
Epoch 6/10
 - 9s - loss: 0.7093 - acc: 0.7589 - val_loss: 1.4874 - val_acc: 0.5148
Epoch 7/10
 - 9s - loss: 0.5934 - acc: 0.8040 - val_loss: 1.6000 - val_acc: 0.5149
Epoch 8/10
 - 8s - loss: 0.4784 - acc: 0.8417 - val_loss: 1.7071 - val_acc: 0.5178
Epoch 9/10
 - 8s - loss: 0.3779 - acc: 0.8825 - val_loss: 1.8911 - val_acc: 0.5058
Epoch 10/10
 - 8s - loss: 0.2950 - acc: 0.9082 - val_loss: 2.0192 - val_acc: 0.5058

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 32, 32)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 30, 30)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 14, 14)        4640      
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                62730     
Total params: 67,818
Trainable params: 67,818
Non-trainable params: 0
_________________________________________________________________


## Opdracht
Wat we graag willen is deze notebook uitgewerkt in een package met het cookiecutter template. We willen dan graag een splitsing tussen het trainingsdeel en het scoring deel. 

Het trainingsdeel levert een model en eventuele metadata op (opgeslagen op disk).  
Het scoringsdeel gebruikt het model om de testset te predicten.  
Runnen met: `python <filename> 'scoring'` of `python <filename> 'training'`

Gebruik goede error handling voor bijvoorbeeld het predicten zonder model, of een verkeerd argument meegeven.